In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

In [2]:
url_front = 'http://data.insideairbnb.com/singapore/sg/singapore/'
url_end_listings = '/data/listings.csv.gz'
url_end_reviews = '/data/reviews.csv.gz'

# Combine Data

In [3]:
dates = ['2019-03-18', '2019-04-20', '2019-05-23', '2019-06-25', '2019-07-21', '2019-08-28', '2019-09-25', '2019-10-25', '2019-11-26', '2019-12-28',
         '2020-01-26', '2020-02-27', '2020-03-21', '2020-04-26', '2020-05-27', '2020-06-22', '2020-10-26', '2020-11-27', '2020-12-29','2021-01-27']

In [4]:
# Keep common features across all months (from Oct, 2020 onwards some more features were added)
listings_2019_03 = pd.read_csv(url_front+'2019-03-18'+url_end_listings)
listings_2020_10 = pd.read_csv(url_front+'2020-10-26'+url_end_listings)

In [5]:
long_features = listings_2019_03.columns.values
short_features = listings_2020_10.columns.values
keep = [x for x in long_features if x in short_features]

In [6]:
keep

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30',
 'availability_60',
 'avail

In [7]:
delete_col = ['listing_url',
 'scrape_id',
 'last_scraped',
 'picture_url',
 'host_url',
 'host_name',
 'host_location',
 'host_acceptance_rate',
 'host_thumbnail_url',
 'host_picture_url',
 'host_listings_count',
 'host_total_listings_count',
 'neighbourhood',
 'has_availability',
 'calendar_last_scraped',
 'first_review',
 'last_review',
 'license',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms']

In [8]:
def read_dataset(date):
    reviews = pd.read_csv(url_front+date+url_end_reviews).drop(columns = ['id'])
    listings = pd.read_csv(url_front+date+url_end_listings)[keep].drop(columns = delete_col) 

    return reviews, listings

In [57]:
def anti_join(x, y, date, on):
    # Find comments for new listings.
    merged = pd.merge(left=x, right=y, how='left', indicator=True, on=on)
    left_only = merged.loc[merged._merge == 'left_only', :].drop(columns='_merge')
    ans_1 = left_only.drop(
        columns = ['date_y', 'reviewer_id_y', 'reviewer_name_y','comments_y']).rename(
        columns = lambda x: x.replace('_x', ''))
    
    # Find new comments for existing listings.
    left_right = merged.loc[(merged._merge == 'both') & (merged.date_x >= date), :].drop(columns='_merge')
    ans_2 = left_right.drop(
        columns = ['date_y', 'reviewer_id_y', 'reviewer_name_y','comments_y']).rename(
        columns = lambda x: x.replace('_x', ''))
    
    ans = pd.concat([ans_1,ans_2]).drop_duplicates()
    return ans

In [58]:
reviews, listings = read_dataset(dates[0])
data = pd.merge(reviews, listings, left_on = 'listing_id', right_on = 'id', how = 'left').drop(columns = 'id')   

In [59]:
for i in range (1, 19):
    date = dates[i]
    this_review, this_listings = read_dataset(date)
    new_review = anti_join(this_review, reviews, dates[i-1], 'listing_id')
    new_review = new_review.drop_duplicates()
    new_data = pd.merge(new_review, this_listings, left_on = 'listing_id', right_on = 'id', how = 'left').drop(columns = 'id') 
    
    # concat the data 
    reviews = pd.concat([reviews, new_review]).drop_duplicates()
    data = pd.concat([data, new_data]).drop_duplicates()
    
    print('finish', i)

finish 1


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (42,43,61,62,94,95) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


finish 2
finish 3
finish 4
finish 5
finish 6
finish 7
finish 8
finish 9
finish 10
finish 11
finish 12
finish 13
finish 14
finish 15
finish 16
finish 17
finish 18


In [60]:
data.comments.isna().value_counts()

False    155078
True        172
Name: comments, dtype: int64

In [61]:
reviewer_id = data.pop('reviewer_id')
data.insert(0, 'reviewer_id', reviewer_id)
data.head()

,reviewer_id,listing_id,date,reviewer_name,comments,name,description,neighborhood_overview,host_id,host_since,host_about,host_response_time,host_response_rate,host_is_superhost,host_neighbourhood,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month
0,8557223,49091,2013-10-21,Jared,Fran was absolutely gracious and welcoming. Ma...,COZICOMFORT LONG TERM STAY ROOM 2,This is Room No. 2.(available after Jan 15 201...,NaN,266763,2010-10-20,I am a private tutor by profession. My husband...,within a few hours,100%,f,Woodlands,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,f,Woodlands,North Region,1.44255,103.79580,Apartment,Private room,1,1.0,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$81.00,180,360,180,180,360,360,180.0,360.0,58 months ago,30,60,90,365,1,0,94.0,10.0,10.0,10.0,10.0,8.0,8.0,f,2,0.02
1,1356099,50646,2014-04-18,James,A comfortable room in a smart condo developmen...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,"I am a working professional, living in Singapo...",NaN,NaN,f,Bukit Timah,"['email', 'phone', 'facebook', 'reviews']",t,f,Bukit Timah,Central Region,1.33235,103.78521,Apartment,Private room,2,1.0,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$80.00,90,730,90,90,730,730,90.0,730.0,55 months ago,30,60,90,365,18,0,91.0,9.0,10.0,10.0,10.0,9.0,9.0,f,1,0.30
2,15222393,50646,2014-06-05,Welli,Stayed over at Sujatha's house for 3 good nigh...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,"I am a working professional, living in Singapo...",NaN,NaN,f,Bukit Timah,"['email', 'phone', 'facebook', 'reviews']",t,f,Bukit Timah,Central Region,1.33235,103.78521,Apartment,Private room,2,1.0,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$80.00,90,730,90,90,730,730,90.0,730.0,55 months ago,30,60,90,365,18,0,91.0,9.0,10.0,10.0,10.0,9.0,9.0,f,1,0.30
3,5543172,50646,2014-07-02,Cyril,It's been a lovely stay at Sujatha's. The room...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,"I am a working professional, living in Singapo...",NaN,NaN,f,Bukit Timah,"['email', 'phone', 'facebook', 'reviews']",t,f,Bukit Timah,Central Region,1.33235,103.78521,Apartment,Private room,2,1.0,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$80.00,90,730,90,90,730,730,90.0,730.0,55 months ago,30,60,90,365,18,0,91.0,9.0,10.0,10.0,10.0,9.0,9.0,f,1,0.30
4,817532,50646,2014-07-08,Jake,"We had a great experience. A nice place, an am...",Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,"I am a working professional, living in Singapo...",NaN,NaN,f,Bukit Timah,"['email', 'phone', 'facebook', 'reviews']",t,f,Bukit Timah,Central Region,1.33235,103.78521,Apartment,Private room,2,1.0,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$80.00,90,730,90,90,730,730,90.0,730.0,55 months ago,30,60,90,365,18,0,91.0,9.0,10.0,10.0,10.0,9.0,9.0,f,1,0.30


# Find Nearby Sites

In [ ]:
# Extract the location data.
df_lat_lon = data[['latitude','longitude']]

In [ ]:
# Read in the 